# The Functional API

## Single Input and Output: Multiclass Classification Example

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Create a simple model.
inputs = keras.Input(shape=(3,), name="my_input")
features = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10, activation="softmax")(features)

# Instantiate the model by specifying the inputs and outputs.
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

## Multiple Inputs and Outputs: Customer Support Tickets Example

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
# Create a model with multiple inputs and outputs.
# Inputs: title of the ticket, text body of the ticket, and any tags added by the user
# Outputs: priority score and department assigned

vocabulary_size = 10000
num_tags = 100
num_departments = 4

# Define our input layers
title = keras.Input(
    shape=(vocabulary_size,), name="title"
)  # Variable-length sequence of ints
text_body = keras.Input(shape=(vocabulary_size,), name="body")  # Variable-length sequence of ints
tags = keras.Input(
    shape=(num_tags,), name="tags"
)  # Binary vectors of size `num_tags`

# Concatenate inputs into a single feature vector
features = layers.Concatenate()(
    [title, text_body, tags]
)  # Features are concatenated
features = layers.Dense(64, activation="relu")(features) # Project to 64-dimensions for a richer representation

# We project onto a single unit output layer, and squash it with a sigmoid:
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(num_departments, activation="softmax", name="department")(
    features
)

# Now we define the model we will train
model = keras.Model(
    inputs=[title, text_body, tags],
    outputs=[priority, department],
)

# Compile the model with losses and metrics
model.compile(
    optimizer="rmsprop",
    loss=["mean_squared_error", "categorical_crossentropy"],
    metrics=[["mean_absolute_error"], ["accuracy"]],
)

# Train the model by passing lists of NumPy arrays of inputs and targets
# since we have multiple inputs and outputs

num_samples = 1280
title_data = np.random.randint(
    0, 2, size=(num_samples, vocabulary_size)
)  # Dummy title data

body_data = np.random.randint(
    0, 2, size=(num_samples, vocabulary_size)
)  # Dummy body data

tags_data = np.random.randint(0, 2, size=(num_samples, num_tags)).astype("float32")  # Dummy tags data
priority_data = np.random.random(size=(num_samples, 1))  # Dummy priority data
department_data = np.random.randint(
    0, 2, size=(num_samples, num_departments)
)  # Dummy department data

model.fit(
    {"title": title_data, "body": body_data, "tags": tags_data},
    {"priority": priority_data, "department": department_data},
    epochs=2,
    batch_size=32,
)

model.evaluate(
    {"title": title_data, "body": body_data, "tags": tags_data},
    {"priority": priority_data, "department": department_data},
    batch_size=32,
)

model.predict(
    {"title": title_data, "body": body_data, "tags": tags_data},
    batch_size=32,
)

## Using Feature Extraction to Create a New Model: Reusing Intermediate Layer Outputs Example

In [ ]:
# Create a new model by reusing intermediate layer outputs
features = model.layers[4].output
difficulty = layers.Dense(1, activation="sigmoid", name="difficulty")(features)
new_model = keras.Model(
    inputs=model.inputs, outputs=[model.outputs[0], difficulty]
)